## Imports

In [45]:


import spacy
import nltk

import pandas as pd


from sklearn.model_selection import train_test_split

import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.feature_selection import RFE
from sklearn.svm import SVR
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE, RandomOverSampler
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
random_seed = 42
from xgboost import XGBClassifier

## data

### read

In [46]:
df_features = pd.read_csv('../data/processed/processed_dataset.csv')
df_features

,corpus_name,id,parent,child,label,dep,parent_text,parent_lemma,parent_pos,parent_tag,...,child_lemma,child_pos,child_tag,child_shape,child_is_alpha,child_is_stop,child_morph,child_ner,child_ner_start_end,parent_ner_start_end
0,sci,1,ter,person,:ARG0,nsubj,tem,ter,VERB,VERB,...,Meyer,PROPN,PROPN,Xxxxx,True,False,Gender=Masc|Number=Sing,PER,"(0, 1)",NaN
1,sci,4,fazer,isso,:ARG1,nao_tem_dep,feito,fazer,VERB,VERB,...,isso,PRON,PRON,Xxxx,True,True,Gender=Masc|Number=Sing|PronType=Dem,NaN,NaN,NaN
2,sci,4,fazer,universidade,:ARG3,obl,feito,fazer,VERB,VERB,...,universidade,NOUN,NOUN,xxxx,True,False,Gender=Fem|Number=Sing,NaN,NaN,NaN
3,sci,12,acontecer,isso,:ARG1,nsubj,acontece,acontecer,VERB,VERB,...,isso,PRON,PRON,xxxx,True,True,Gender=Masc|Number=Sing|PronType=Dem,NaN,NaN,NaN
4,sci,16,entender,otimista,:ARG0,nsubj,entendem,entender,VERB,VERB,...,otimista,NOUN,NOUN,xxxx,True,False,Gender=Masc|Number=Plur,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3065,news,poder-4-156,dizer,ter,:ARG1,ccomp,disse,dizer,VERB,VERB,...,ter,VERB,VERB,xxx,True,True,Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbF...,NaN,NaN,NaN
3066,news,poder-4-156,ter,ponto,:ARG1,obj,tem,ter,VERB,VERB,...,ponto,NOUN,NOUN,xxxx,True,True,Gender=Masc|Number=Plur,NaN,NaN,NaN
3067,news,ilustrissima-4-167,utilizar,referência,:ARG1,obj,utilizar,utilizar,VERB,VERB,...,referência,NOUN,NOUN,xxxx,True,False,Gender=Fem|Number=Plur,NaN,NaN,NaN
3068,news,ciencia-4-171,chamar,fenômeno,:ARG1,obj,chamar,chamar,VERB,VERB,...,fenômeno,NOUN,NOUN,xxxx,True,False,Gender=Masc|Number=Sing,NaN,NaN,NaN


In [47]:
### all nan to nsa (nao se aplica)
df_features.fillna('nsa', inplace = True)

In [48]:
df_features.label.value_counts()

label
:ARG1        1517
:ARG0        1033
:ARG2         296
:ARG1-of       95
:ARG0-of       63
:ARG3          37
:ARG4          23
:ARG2-of        5
:ARG01-of       1
Name: count, dtype: int64

In [49]:
df_features['label'] = df_features.label.apply(
    lambda x: 
        0 if "arg0" in x.casefold() else
        1 if "arg1" in x.casefold() else
        2 if "arg2" in x.casefold() else

        # 3 if "arg3" in x.casefold() else
        # 3 if "arg4" in x.casefold() else
        # 3 if "arg5" in x.casefold() else
        # "erro"
        
        3 if "arg3" in x.casefold() else
        4 if "arg4" in x.casefold() else
        5 if "arg5" in x.casefold() else
        "erro"
        
)

df_features['label'].value_counts()

label
1    1612
0    1097
2     301
3      37
4      23
Name: count, dtype: int64

In [50]:
df_features = df_features[
    (df_features.label != 'erro') & (df_features.label != 5)]
df_features.label.value_counts()

label
1    1612
0    1097
2     301
3      37
4      23
Name: count, dtype: int64

In [51]:
df_features.label = df_features.label.astype('int')

In [52]:
df_features['stratify-column'] = df_features.apply(lambda x: str(x.corpus_name) + "_" + str(x.label), axis = 1)
df_features['stratify-column']

0        sci_0
1        sci_1
2        sci_3
3        sci_1
4        sci_0
         ...  
3065    news_1
3066    news_1
3067    news_1
3068    news_1
3069    news_2
Name: stratify-column, Length: 3070, dtype: object

### split

In [53]:
df_features.columns

Index(['corpus_name', 'id', 'parent', 'child', 'label', 'dep', 'parent_text',
       'parent_lemma', 'parent_pos', 'parent_tag', 'parent_shape',
       'parent_is_alpha', 'parent_is_stop', 'parent_morph', 'parent_ner',
       'child_text', 'child_lemma', 'child_pos', 'child_tag', 'child_shape',
       'child_is_alpha', 'child_is_stop', 'child_morph', 'child_ner',
       'child_ner_start_end', 'parent_ner_start_end', 'stratify-column'],
      dtype='object')

In [54]:
X = df_features.drop(['corpus_name', 'id', 'stratify-column', "label"], axis = 1)
y = df_features['label']

In [55]:
X_enc = pd.get_dummies(X)
X_enc

,parent_is_alpha,parent_is_stop,child_is_alpha,child_is_stop,parent_abandonar,parent_abismo,parent_abordar,parent_abrir,parent_acabar,parent_academia,...,"parent_ner_start_end_(0, 1)","parent_ner_start_end_(0, 3)","parent_ner_start_end_(0, 5)","parent_ner_start_end_(1, 2)","parent_ner_start_end_(1, 4)","parent_ner_start_end_(11, 12)","parent_ner_start_end_(2, 3)","parent_ner_start_end_(4, 5)","parent_ner_start_end_(7, 8)",parent_ner_start_end_nsa
0,True,True,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
1,True,False,True,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
2,True,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
3,True,False,True,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
4,True,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3065,True,False,True,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
3066,True,True,True,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
3067,True,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True
3068,True,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,True


In [56]:
X_train, X_test, y_train, y_test = train_test_split(X_enc, y, 
                                                    test_size=0.20, 
                                                    random_state=42, 
                                                    stratify=y#df_features['stratify-column']
                                                    )

In [57]:
import cupy as cp

In [58]:
# import scipy.stats as st

# selector = RFE(RandomForestClassifier(random_state=random_seed), step=100, verbose = 2)
# sampler = SMOTE(random_state=random_seed)
# scaler = StandardScaler(with_mean=False)

# estimator = XGBClassifier(
#     device = 'cuda',
# #    tree_method = 'hist'
#     )

# pipeline_steps = [
#     ('scaling', scaler),
#     ('sampling', sampler),
#     ('selecting', selector),
#     ('estimator',estimator)
#     ] 



# pipeline = Pipeline(steps = pipeline_steps)


# # Preconfigure estimator and parameters
# params = {  
#             "estimator__n_estimators": st.randint(3, 40),
#             "estimator__max_depth": st.randint(3, 40),
#             "estimator__learning_rate": st.uniform(0.05, 0.4),
#             "estimator__colsample_bytree": st.beta(10, 1),
#             "estimator__subsample": st.beta(10, 1),
#             "estimator__gamma": st.uniform(0, 10),
#             'estimator__objective': ['multi:softmax'],
#             'estimator__eval_metric':['aucpr'],
            
# #            'estimator__scale_pos_weight': st.randint(0, 2),
#             "estimator__min_child_weight": st.expon(0, 50),
#         }

# # Random Search Training with 5 folds Cross Validation
# rf_random = RandomizedSearchCV(
#     pipeline, 
#     params, 
#     cv=5, 
#     verbose=4,
#     #n_jobs=-1, 
#     n_iter=50, 
#     random_state=42, 
#     scoring='f1_macro') 
                              
# rf_random.fit(X_train, y_train)  
# best_params = rf_random.best_params_
# best_score = rf_random.best_score_

# print('best_params', best_params)
# print('best_score', best_score)

# model = rf_random.best_estimator_

In [59]:
#################################################
# teste
#################################################


selector = RFE(RandomForestClassifier(random_state=random_seed), step=100, verbose = 2)
sampler = RandomOverSampler(random_state=random_seed)
scaler = StandardScaler(with_mean=False)

estimator = XGBClassifier(
    device = 'cuda',
#    tree_method = 'hist'
    )

pipeline_steps = [
    ('scaling', scaler),
    ('sampling', sampler),
    ('selecting', selector),
    ('estimator',estimator)
    ] 



pipeline = Pipeline(steps = pipeline_steps)



pipeline.fit(X_train, y_train)
model = pipeline


Fitting estimator with 6443 features.
Fitting estimator with 6343 features.
Fitting estimator with 6243 features.
Fitting estimator with 6143 features.
Fitting estimator with 6043 features.


KeyboardInterrupt: 

In [ ]:
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- child
- child_lemma
- child_morph
- child_ner
- child_ner_start_end
- ...
Feature names seen at fit time, yet now missing:
- child_"s/n"
- child_1.578
- child_5
- child_CRM
- child_Comandante
- ...


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.76      0.80      0.78       220
           1       0.81      0.73      0.77       322
           2       0.32      0.43      0.37        60
           3       0.14      0.14      0.14         7
           4       0.50      0.20      0.29         5

    accuracy                           0.71       614
   macro avg       0.51      0.46      0.47       614
weighted avg       0.73      0.71      0.72       614



## interpretabilidade

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
importances = model.steps[-1][1].feature_importances_

feature_scores = pd.Series(importances, index=X_train.columns).sort_values(ascending=False)
feature_scores

In [ ]:
feature_scores = feature_scores.reset_index().rename({'index':'column_name_ohc', 0:'feature_score'},axis =1)
feature_scores

In [ ]:
feature_scores['original_column'] = feature_scores.column_name_ohc.apply(lambda x: '_'.join(x.split('_')[:-1]))
feature_scores['original_column'].value_counts()

In [ ]:
feature_scores[['original_column', 'feature_score']].groupby('original_column').sum().sort_values('feature_score')

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
cm=confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='g') # font size

In [ ]:
df_features_test = df_features.loc[y_test.index.tolist(), :]

In [ ]:
df_anl = pd.DataFrame({
    "y_test": y_test.tolist(),
    "y_pred": y_pred.tolist(),
    "corpus_name": df_features.loc[y_test.index.tolist(), :]['corpus_name'].tolist(),  
})

In [ ]:
for cp_nm in df_anl.corpus_name.unique():
    
    print(f"""###############################################
# corpus name: {cp_nm}
###############################################
          """)
    
    df_anl_cp = df_anl[df_anl.corpus_name == cp_nm]
    
    print(classification_report(df_anl_cp.y_test, df_anl_cp.y_pred))
    
    cm=confusion_matrix(df_anl_cp.y_test, df_anl_cp.y_pred)
    sns.heatmap(cm, annot=True, fmt='g') # font size
    plt.show()

In [ ]:
y_test_s = y_test.copy()
y_test_s['corpus']

In [ ]:
n = 10

f, ax = plt.subplots(figsize=(30, 24))
ax = sns.barplot(x=feature_scores[:n], y=feature_scores.index[:n])
ax.set_title("Visualize feature scores of the features")
ax.set_yticklabels(feature_scores.index)
ax.set_xlabel("Feature importance score")
ax.set_ylabel("Features")
plt.show()